In [22]:
import pandas as pd
faltas = pd.read_csv('../data/processed/analise_faltas.csv')
faltas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20953 entries, 0 to 20952
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   partida_id  20953 non-null  int64 
 1   data        20953 non-null  object
 2   vencedor    20953 non-null  object
 3   mandante    20953 non-null  object
 4   visitante   20953 non-null  object
 5   clube       20953 non-null  object
 6   cartao      20953 non-null  object
 7   atleta      20947 non-null  object
 8   posicao     19755 non-null  object
 9   minuto      20953 non-null  object
 10  resultado   20953 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.8+ MB


In [23]:
# extrai minuto numérico (trata valores como '90+')
faltas['minuto_num'] = pd.to_numeric(faltas['minuto'].astype(str).str.extract(r'(\d+)')[0], errors='coerce')

# cria faixas de tempo de jogo (11 bins: 10 min + 45+ + 90+)
def categorize_bins(row):
    m_str = str(row['minuto'])
    m_num = row['minuto_num']
    
    if pd.isna(m_num): return 'Desconhecido'
    
    if '45+' in m_str: return '45+'
    if '90+' in m_str or m_num > 90: return '90+'
    
    if m_num <= 10: return '00-10'
    if m_num <= 20: return '11-20'
    if m_num <= 30: return '21-30'
    if m_num <= 40: return '31-40'
    if m_num <= 45: return '41-45'
    if m_num <= 60: return '46-60'
    if m_num <= 70: return '61-70'
    if m_num <= 80: return '71-80'
    return '81-90'

faltas['tempo_jogo'] = faltas.apply(categorize_bins, axis=1)

# Ordenar categorias para garantir a ordem correta nos gráficos
cats = ['00-10', '11-20', '21-30', '31-40', '41-45', '45+', '46-60', '61-70', '71-80', '81-90', '90+']
faltas['tempo_jogo'] = pd.Categorical(faltas['tempo_jogo'], categories=cats, ordered=True)

# padroniza posicao nula
faltas['posicao'] = faltas['posicao'].fillna('Desconhecido')
# extrai minuto numérico (trata valores como '90+')
faltas['minuto_num'] = pd.to_numeric(faltas['minuto'].astype(str).str.extract(r'(\d+)')[0], errors='coerce')

In [24]:
faltas.tempo_jogo.value_counts()

tempo_jogo
81-90    3165
46-60    2919
71-80    2686
61-70    2464
90+      2350
31-40    2093
21-30    1585
41-45    1180
11-20    1103
45+       859
00-10     549
Name: count, dtype: int64

In [25]:
# Substituir zagueira por zagueiro
faltas['posicao'] = faltas['posicao'].replace('Zagueira', 'Zagueiro')

In [26]:
faltas.posicao.value_counts()

posicao
Meio-campo      8697
Zagueiro        7482
Atacante        2874
Desconhecido    1198
Goleiro          702
Name: count, dtype: int64

In [27]:
# cria dataset agrupado
faltas_grouped = (
    faltas
    .groupby(['cartao', 'posicao', 'tempo_jogo', 'clube'])
    .size()
    .reset_index(name='ocorrencias')
    .sort_values(['ocorrencias'], ascending=False)
)

C:\Users\david\AppData\Local\Temp\ipykernel_19780\2066270223.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['cartao', 'posicao', 'tempo_jogo', 'clube'])


In [28]:
# Soma cartões amarelos e vermelhos
faltas_grouped = (
    faltas_grouped
    .groupby(['posicao', 'tempo_jogo', 'clube'], observed=False)['ocorrencias']
    .sum()
    .reset_index(name='Quantidade_Cartoes')
)
# exibe as primeiras linhas do agrupamento
faltas_grouped[50::-2].head()

,posicao,tempo_jogo,clube,Quantidade_Cartoes
50,Atacante,11-20,Figueirense,1
48,Atacante,11-20,Cruzeiro,4
46,Atacante,11-20,Coritiba,7
44,Atacante,11-20,Chapecoense,2
42,Atacante,11-20,CSA,0


In [29]:
faltas_grouped.to_csv('../data/processed/analise_cartoes.csv', index=False)

In [30]:
faltas_grouped["Quantidade_Cartoes"].value_counts().sort_index()

Quantidade_Cartoes
0     357
1     183
2     130
3      89
4      87
     ... 
62      1
67      1
68      1
72      1
73      1
Name: count, Length: 67, dtype: int64